<a href="https://colab.research.google.com/github/sujay-dsa/ComputerVision/blob/master/EVA_5_Session_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input -? OUtput? RF
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(256, 512, 3)
        self.conv6 = nn.Conv2d(512, 1024, 3)
        self.conv7 = nn.Conv2d(1024, 10, 3)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = F.relu(self.conv7(x))
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 15, 3, padding=1) #input -? OUtput? RF
        self.batchn1=nn.BatchNorm2d(15)
        self.conv2 = nn.Conv2d(15, 20, 3, padding=1)

        self.batchnn2=nn.BatchNorm2d(20)
        #self.combine1=nn.Conv1d(32,10,kernel_size=1,stride=1)
        self.pool1 = nn.MaxPool2d(2, 2)

        self.conv3 = nn.Conv2d(20, 23, 3, padding=1)
        self.batchnn3=nn.BatchNorm2d(23)
        self.conv4 = nn.Conv2d(23, 25, 3, padding=1)

        self.batchnn4=nn.BatchNorm2d(25)
        #self.combine2=nn.Conv1d(128,10,kernel_size=1,stride=1)
        self.pool2 = nn.MaxPool2d(2, 2)

        self.conv5 = nn.Conv2d(25, 16, 3)
        self.batchn5=nn.BatchNorm2d(16)
        self.conv6 = nn.Conv2d(16, 16, 3)
        self.batchnn6=nn.BatchNorm2d(16)
        self.conv7 = nn.Conv2d(16, 10, 3)

    def forward(self, x):
        x = self.batchnn2(self.conv2(F.relu(self.batchn1(self.conv1(x)))))
        #x=self.combine1(x)
        x=self.pool1(F.relu(x))
        x = self.batchnn4(self.conv4(F.relu(self.batchnn3(self.conv3(x)))))
        #x=self.combine2(x)
        x=self.pool2(F.relu(x))
        x = self.batchnn6(self.conv6(F.relu(self.batchn5(self.conv5(x)))))
        x = self.conv7(F.relu(x))
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [3]:
#!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 15, 28, 28]             150
       BatchNorm2d-2           [-1, 15, 28, 28]              30
            Conv2d-3           [-1, 20, 28, 28]           2,720
       BatchNorm2d-4           [-1, 20, 28, 28]              40
         MaxPool2d-5           [-1, 20, 14, 14]               0
            Conv2d-6           [-1, 23, 14, 14]           4,163
       BatchNorm2d-7           [-1, 23, 14, 14]              46
            Conv2d-8           [-1, 25, 14, 14]           5,200
       BatchNorm2d-9           [-1, 25, 14, 14]              50
        MaxPool2d-10             [-1, 25, 7, 7]               0
           Conv2d-11             [-1, 16, 5, 5]           3,616
      BatchNorm2d-12             [-1, 16, 5, 5]              32
           Conv2d-13             [-1, 16, 3, 3]           2,320
      BatchNorm2d-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [12]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.RandomRotation(30),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [13]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        #pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [14]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.02, momentum=0.9)

for epoch in range(1, 19):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)


  0%|          | 0/469 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.

100%|██████████| 469/469 [00:21<00:00, 21.42it/s]

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0617, Accuracy: 9803/10000 (98.030%)




100%|██████████| 469/469 [00:22<00:00, 21.29it/s]

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0545, Accuracy: 9822/10000 (98.220%)




100%|██████████| 469/469 [00:21<00:00, 21.39it/s]

  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0195, Accuracy: 9941/10000 (99.410%)




100%|██████████| 469/469 [00:21<00:00, 21.38it/s]

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0221, Accuracy: 9927/10000 (99.270%)




100%|██████████| 469/469 [00:21<00:00, 21.68it/s]



Test set: Average loss: 0.0218, Accuracy: 9929/10000 (99.290%)

